In [3]:
pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 101.6 kB/s eta 0:00:0000:010:03m

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install imgaug

  Using cached imgaug-0.4.0-py2.py3-none-any.whl (948 kB)
  Using cached shapely-2.0.1-cp310-cp310-macosx_11_0_arm64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""
    for dirname, _, filenames in os.walk('./ck'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        hog_descriptors = hog_extraction(extracted_face)
        try:  
            # Calculate the Z-scores for each descriptor
            zscore_hog_descriptors = (hog_descriptors - np.mean(hog_descriptors)) / np.std(hog_descriptors)
            # Fusion using weighted sum
            hog_descriptors = weight2 * zscore_hog_descriptors   
        except Exception as e:
            # print(str(e), np.std(hog_descriptors))
            pass    
        hog_descriptors = np.resize(hog_descriptors, (desired_length,))

        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))

        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        sift_descriptors = sift_extraction(extracted_face)
        try:
            # Calculate the Z-scores for each descriptor
            zscore_sift_descriptors = (sift_descriptors - np.mean(sift_descriptors)) / np.std(sift_descriptors)
            # Fusion using weighted sum
            sift_descriptors = weight4 * zscore_sift_descriptors   
        except Exception as e:
            print(str(e))
            pass    
        sift_descriptors = np.resize(sift_descriptors, (desired_length,))

        fused_features = sift_descriptors
        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, orb_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def hog_extraction(extracted_face):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

    # Define HOG parameters
    orientations = 9  # Number of gradient orientations
    pixels_per_cell = (8, 8)  # Cell size in pixels
    cells_per_block = (2, 2)  # Number of cells in each block

    # Compute HOG features
    hog_image, hog_features = hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)    
    # print(hog_features)
    disply_feature("HOG", hog_features)
    return hog_features

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

# Scale-Invariant Feature Transform descriptor
def sift_extraction(extracted_face):
    # Create a SIFT object
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None)
    # print(descriptors)
    disply_feature("SIFT", image_with_keypoints)
    return descriptors

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
 
    # Reshape the fused features to have two dimensions
    fused_features = fused_features.reshape(fused_features.shape[0], -1)

    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.2, random_state=42)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"sift_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_ck_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


unsupported operand type(s) for /: 'NoneType' and 'int'
Accuracy: 0.6294416243654822
Precision: 0.5993255331822338
Recall: 0.5677750271420349
F1 Score: 0.5758044526417071


In [3]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""
    for dirname, _, filenames in os.walk('./ck'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        hog_descriptors = hog_extraction(extracted_face)
        try:  
            # Calculate the Z-scores for each descriptor
            zscore_hog_descriptors = (hog_descriptors - np.mean(hog_descriptors)) / np.std(hog_descriptors)
            # Fusion using weighted sum
            hog_descriptors = weight2 * zscore_hog_descriptors   
        except Exception as e:
            # print(str(e), np.std(hog_descriptors))
            pass    
        hog_descriptors = np.resize(hog_descriptors, (desired_length,))

        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))

        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        sift_descriptors = sift_extraction(extracted_face)
        try:
            # Calculate the Z-scores for each descriptor
            zscore_sift_descriptors = (sift_descriptors - np.mean(sift_descriptors)) / np.std(sift_descriptors)
            # Fusion using weighted sum
            sift_descriptors = weight4 * zscore_sift_descriptors   
        except Exception as e:
            print(str(e))
            pass    
        sift_descriptors = np.resize(sift_descriptors, (desired_length,))

        fused_features = hog_descriptors
        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, orb_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def hog_extraction(extracted_face):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

    # Define HOG parameters
    orientations = 9  # Number of gradient orientations
    pixels_per_cell = (8, 8)  # Cell size in pixels
    cells_per_block = (2, 2)  # Number of cells in each block

    # Compute HOG features
    hog_image, hog_features = hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)    
    # print(hog_features)
    disply_feature("HOG", hog_features)
    return hog_features

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

# Scale-Invariant Feature Transform descriptor
def sift_extraction(extracted_face):
    # Create a SIFT object
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None)
    # print(descriptors)
    disply_feature("SIFT", image_with_keypoints)
    return descriptors

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
 
    # Reshape the fused features to have two dimensions
    fused_features = fused_features.reshape(fused_features.shape[0], -1)

    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.2, random_state=42)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"hog_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_ck_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


unsupported operand type(s) for /: 'NoneType' and 'int'
Accuracy: 0.9441624365482234
Precision: 0.9208335910121624
Recall: 0.9272525481764612
F1 Score: 0.9226172147687494


In [19]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""
    for dirname, _, filenames in os.walk('./ck'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        hog_descriptors = hog_extraction(extracted_face)
        try:  
            # Calculate the Z-scores for each descriptor
            zscore_hog_descriptors = (hog_descriptors - np.mean(hog_descriptors)) / np.std(hog_descriptors)
            # Fusion using weighted sum
            hog_descriptors = weight2 * zscore_hog_descriptors   
        except Exception as e:
            # print(str(e), np.std(hog_descriptors))
            pass    
        hog_descriptors = np.resize(hog_descriptors, (desired_length,))

        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))

        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        sift_descriptors = sift_extraction(extracted_face)
        try:
            # Calculate the Z-scores for each descriptor
            zscore_sift_descriptors = (sift_descriptors - np.mean(sift_descriptors)) / np.std(sift_descriptors)
            # Fusion using weighted sum
            sift_descriptors = weight4 * zscore_sift_descriptors   
        except Exception as e:
            print(str(e))
            pass    
        sift_descriptors = np.resize(sift_descriptors, (desired_length,))

        # fused_features = hog_descriptors
        fused_features = np.concatenate((hog_descriptors, lbp_descriptors, orb_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def hog_extraction(extracted_face):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

    # Define HOG parameters
    orientations = 9  # Number of gradient orientations
    pixels_per_cell = (8, 8)  # Cell size in pixels
    cells_per_block = (2, 2)  # Number of cells in each block

    # Compute HOG features
    hog_image, hog_features = hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)    
    # print(hog_features)
    disply_feature("HOG", hog_features)
    return hog_features

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

# Scale-Invariant Feature Transform descriptor
def sift_extraction(extracted_face):
    # Create a SIFT object
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None)
    # print(descriptors)
    disply_feature("SIFT", image_with_keypoints)
    return descriptors

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
 
    # Reshape the fused features to have two dimensions
    fused_features = fused_features.reshape(fused_features.shape[0], -1)

    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.2, random_state=42)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(" 4 algorithm ")
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"4_algo_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_ck_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


unsupported operand type(s) for /: 'NoneType' and 'int'
 4 algorithm 
Accuracy: 0.949238578680203
Precision: 0.9572955115060379
Recall: 0.923334551211789
F1 Score: 0.9368567510769232


In [46]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""
    augmentation_seq = iaa.Sequential([
        # iaa.Fliplr(),  # Flip images horizontally with a 50% chance
        # iaa.Multiply((0.8, 1.2)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        iaa.Affine(rotate=(0, 10)),  # Rotate images by -10 to +10 degrees
        # iaa.GaussianBlur(sigma=(0, 1.0)),  # Apply Gaussian blur with a sigma between 0 and 1.0
        # iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add Gaussian noise with a scale of 0 to 0.05*255
    ])
    for dirname, _, filenames in os.walk('./ck'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                # image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        hog_descriptors = hog_extraction(extracted_face)
        try:  
            # Calculate the Z-scores for each descriptor
            zscore_hog_descriptors = (hog_descriptors - np.mean(hog_descriptors)) / np.std(hog_descriptors)
            # Fusion using weighted sum
            hog_descriptors = weight2 * zscore_hog_descriptors   
        except Exception as e:
            # print(str(e), np.std(hog_descriptors))
            pass    
        hog_descriptors = np.resize(hog_descriptors, (desired_length,))

        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))

        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            # print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        sift_descriptors = sift_extraction(extracted_face)
        try:
            # Calculate the Z-scores for each descriptor
            zscore_sift_descriptors = (sift_descriptors - np.mean(sift_descriptors)) / np.std(sift_descriptors)
            # Fusion using weighted sum
            sift_descriptors = weight4 * zscore_sift_descriptors   
        except Exception as e:
            # print(str(e))
            pass    
        sift_descriptors = np.resize(sift_descriptors, (desired_length,))

        # fused_features = lbp_descriptors + hog_descriptors + sift_descriptors
        fused_features = hog_descriptors
        if lbp_descriptors is not None:
            fused_features += lbp_descriptors
        if sift_descriptors is not None:
            fused_features += sift_descriptors.astype(np.float64)

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def hog_extraction(extracted_face):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

    # Define HOG parameters
    orientations = 9  # Number of gradient orientations
    pixels_per_cell = (8, 8)  # Cell size in pixels
    cells_per_block = (2, 2)  # Number of cells in each block

    # Compute HOG features
    hog_image, hog_features = hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)    
    # print(hog_features)
    disply_feature("HOG", hog_features)
    return hog_features

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

# Scale-Invariant Feature Transform descriptor
def sift_extraction(extracted_face):
    # Create a SIFT object
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None)
    # print(descriptors)
    disply_feature("SIFT", image_with_keypoints)
    return descriptors

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.24, random_state=31)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(" 3_without_orb_algo algorithm ")
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"test_3_without_orb_algo_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_ck_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


 3_without_orb_algo algorithm 
Accuracy: 0.9872881355932204
Precision: 0.9766233766233767
Recall: 0.9794050776809398
F1 Score: 0.9774916387959866


In [47]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""
    augmentation_seq = iaa.Sequential([
        # iaa.Affine(rotate=(0, 10)),  # Rotate images by -10 to +10 degrees
        iaa.Multiply((0.8, 1.0)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(gamma=(0.8, 1.2)),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Fliplr(),  # Flip images horizontally with a 50% chance
        # iaa.Multiply((0.8, 1.2)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Affine(rotate=(0, 10)),  # Rotate images by -10 to +10 degrees
        # iaa.GaussianBlur(sigma=(0, 1.0)),  # Apply Gaussian blur with a sigma between 0 and 1.0
        # iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add Gaussian noise with a scale of 0 to 0.05*255
    ])
    for dirname, _, filenames in os.walk('./ck'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        hog_descriptors = hog_extraction(extracted_face)
        try:  
            # Calculate the Z-scores for each descriptor
            zscore_hog_descriptors = (hog_descriptors - np.mean(hog_descriptors)) / np.std(hog_descriptors)
            # Fusion using weighted sum
            hog_descriptors = weight2 * zscore_hog_descriptors   
        except Exception as e:
            # print(str(e), np.std(hog_descriptors))
            pass    
        hog_descriptors = np.resize(hog_descriptors, (desired_length,))

        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))

        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            # print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        sift_descriptors = sift_extraction(extracted_face)
        try:
            # Calculate the Z-scores for each descriptor
            zscore_sift_descriptors = (sift_descriptors - np.mean(sift_descriptors)) / np.std(sift_descriptors)
            # Fusion using weighted sum
            sift_descriptors = weight4 * zscore_sift_descriptors   
        except Exception as e:
            # print(str(e))
            pass    
        sift_descriptors = np.resize(sift_descriptors, (desired_length,))

        # fused_features = lbp_descriptors + hog_descriptors + sift_descriptors
        fused_features = hog_descriptors
        if lbp_descriptors is not None:
            fused_features += lbp_descriptors
        # if sift_descriptors is not None:
        #     fused_features += sift_descriptors.astype(np.float64)

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def hog_extraction(extracted_face):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

    # Define HOG parameters
    orientations = 9  # Number of gradient orientations
    pixels_per_cell = (8, 8)  # Cell size in pixels
    cells_per_block = (2, 2)  # Number of cells in each block

    # Compute HOG features
    hog_image, hog_features = hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)    
    # print(hog_features)
    disply_feature("HOG", hog_features)
    return hog_features

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

# Scale-Invariant Feature Transform descriptor
def sift_extraction(extracted_face):
    # Create a SIFT object
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None)
    # print(descriptors)
    disply_feature("SIFT", image_with_keypoints)
    return descriptors

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.24, random_state=31)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(" 2_without_orb_sift_algo algorithm ")
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"2_without_orb_sift_algo_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_ck_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


 2_without_orb_sift_algo algorithm 
Accuracy: 0.9957627118644068
Precision: 0.9968253968253968
Recall: 0.9955357142857143
F1 Score: 0.9961272898672577


In [74]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""
    augmentation_seq = iaa.Sequential([
        # iaa.Affine(rotate=(0, 10)),  # Rotate images by -10 to +10 degrees
        iaa.Multiply((0.8, 1.0)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(gamma=(0.8, 1.2)),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Fliplr(),  # Flip images horizontally with a 50% chance
        # iaa.Multiply((0.8, 1.2)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Affine(rotate=(0, 10)),  # Rotate images by -10 to +10 degrees
        # iaa.GaussianBlur(sigma=(0, 1.0)),  # Apply Gaussian blur with a sigma between 0 and 1.0
        # iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add Gaussian noise with a scale of 0 to 0.05*255
    ])
    for dirname, _, filenames in os.walk('./ck'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        hog_descriptors = hog_extraction(extracted_face)
        try:  
            # Calculate the Z-scores for each descriptor
            zscore_hog_descriptors = (hog_descriptors - np.mean(hog_descriptors)) / np.std(hog_descriptors)
            # Fusion using weighted sum
            hog_descriptors = weight2 * zscore_hog_descriptors   
        except Exception as e:
            # print(str(e), np.std(hog_descriptors))
            pass    
        hog_descriptors = np.resize(hog_descriptors, (desired_length,))

        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))

        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            # print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        sift_descriptors = sift_extraction(extracted_face)
        try:
            # Calculate the Z-scores for each descriptor
            zscore_sift_descriptors = (sift_descriptors - np.mean(sift_descriptors)) / np.std(sift_descriptors)
            # Fusion using weighted sum
            sift_descriptors = weight4 * zscore_sift_descriptors   
        except Exception as e:
            # print(str(e))
            pass    
        sift_descriptors = np.resize(sift_descriptors, (desired_length,))

        # fused_features = lbp_descriptors + hog_descriptors + sift_descriptors
        fused_features = hog_descriptors
        if lbp_descriptors is not None:
            fused_features += lbp_descriptors
        if sift_descriptors is not None:
            fused_features += sift_descriptors.astype(np.float64)

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def hog_extraction(extracted_face):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

    # Define HOG parameters
    orientations = 9  # Number of gradient orientations
    pixels_per_cell = (8, 8)  # Cell size in pixels
    cells_per_block = (2, 2)  # Number of cells in each block

    # Compute HOG features
    hog_image, hog_features = hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)    
    # print(hog_features)
    disply_feature("HOG", hog_features)
    return hog_features

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

# Scale-Invariant Feature Transform descriptor
def sift_extraction(extracted_face):
    # Create a SIFT object
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None)
    # print(descriptors)
    disply_feature("SIFT", image_with_keypoints)
    return descriptors

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.1412, random_state=44)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(" 3_without_orb_algo algorithm ")
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"3_without_orb_algo_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_ck_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_ck_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


 3_without_orb_algo algorithm 
Accuracy: 0.9928057553956835
Precision: 0.9947089947089947
Recall: 0.987012987012987
F1 Score: 0.9905018611218072


In [11]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""

    for dirname, _, filenames in os.walk('./jaffe'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                # image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            

        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))


        # fused_features = lbp_descriptors + hog_descriptors + sift_descriptors
        fused_features = lbp_descriptors
        # if lbp_descriptors is not None:
        #     fused_features += lbp_descriptors

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.024, random_state=44)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(" lbp_algo algorithm ", source_data_label)
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"lbp_algo_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


 lbp_algo algorithm  jaffe
Accuracy: 0.6666666666666666
Precision: 0.7142857142857143
Recall: 0.7142857142857143
F1 Score: 0.42857142857142855


In [19]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""

    for dirname, _, filenames in os.walk('./jaffe'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                # image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            

        sift_descriptors = sift_extraction(extracted_face)
        try:
            # Calculate the Z-scores for each descriptor
            zscore_sift_descriptors = (sift_descriptors - np.mean(sift_descriptors)) / np.std(sift_descriptors)
            # Fusion using weighted sum
            sift_descriptors = weight4 * zscore_sift_descriptors   
        except Exception as e:
            # print(str(e))
            pass    
        sift_descriptors = np.resize(sift_descriptors, (desired_length,))

        # fused_features = lbp_descriptors + hog_descriptors + sift_descriptors
        fused_features = sift_descriptors

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

# Scale-Invariant Feature Transform descriptor
def sift_extraction(extracted_face):
    # Create a SIFT object
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None)
    # print(descriptors)
    disply_feature("SIFT", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.024, random_state=44)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print("sift_algo algorithm ", source_data_label)
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"sift_algo_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


sift_algo algorithm  jaffe
Accuracy: 0.3333333333333333
Precision: 0.5
Recall: 0.5714285714285714
F1 Score: 0.23809523809523808


In [20]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""

    for dirname, _, filenames in os.walk('./jaffe'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                # image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            # print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        fused_features = orb_descriptors
        # if orb_descriptors is not None:
        #     fused_features += orb_descriptors.astype(np.float64)

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.024, random_state=44)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print("orb_algo algorithm ", source_data_label)
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"orb_algo_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


orb_algo algorithm  jaffe
Accuracy: 0.16666666666666666
Precision: 0.5
Recall: 0.35714285714285715
F1 Score: 0.07142857142857142


In [32]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""

    for dirname, _, filenames in os.walk('./jaffe'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                # image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        hog_descriptors = hog_extraction(extracted_face)
        try:  
            # Calculate the Z-scores for each descriptor
            zscore_hog_descriptors = (hog_descriptors - np.mean(hog_descriptors)) / np.std(hog_descriptors)
            # Fusion using weighted sum
            hog_descriptors = weight2 * zscore_hog_descriptors   
        except Exception as e:
            # print(str(e), np.std(hog_descriptors))
            pass    
        hog_descriptors = np.resize(hog_descriptors, (desired_length,))

        # fused_features = lbp_descriptors + hog_descriptors + sift_descriptors
        fused_features = hog_descriptors

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def hog_extraction(extracted_face):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

    # Define HOG parameters
    orientations = 9  # Number of gradient orientations
    pixels_per_cell = (8, 8)  # Cell size in pixels
    cells_per_block = (2, 2)  # Number of cells in each block

    # Compute HOG features
    hog_image, hog_features = hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)    
    # print(hog_features)
    disply_feature("HOG", hog_features)
    return hog_features

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.034, random_state=44)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print("hog_algo algorithm ", source_data_label)
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"hog_algo_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


hog_algo algorithm  jaffe
Accuracy: 0.375
Precision: 0.5
Recall: 0.5833333333333334
F1 Score: 0.3333333333333333


In [22]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""
    augmentation_seq = iaa.Sequential([
        # iaa.Affine(rotate=(9, 10)),  # Rotate images by -10 to +10 degrees
        iaa.Multiply((0.8, 1.0)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(gamma=(2.0, 2.2)),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Fliplr(),  # Flip images horizontally with a 50% chance
        # iaa.Multiply((0.8, 1.2)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Affine(rotate=(0, 10)),  # Rotate images by -10 to +10 degrees
        # iaa.GaussianBlur(sigma=(0, 1.0)),  # Apply Gaussian blur with a sigma between 0 and 1.0
        # iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add Gaussian noise with a scale of 0 to 0.05*255
    ])
    for dirname, _, filenames in os.walk('./jaffe'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))

        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            # print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        # fused_features = lbp_descriptors + hog_descriptors + sift_descriptors
        fused_features = lbp_descriptors
        if orb_descriptors is not None:
            fused_features += orb_descriptors.astype(np.float64)

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.024, random_state=44)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(" 2_without_hog_sift_algo algorithm ", source_data_label)
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"2_without_hog_sift_algo_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


 2_without_hog_sift_algo algorithm  jaffe
Accuracy: 0.6666666666666666
Precision: 0.7142857142857143
Recall: 0.7142857142857143
F1 Score: 0.42857142857142855


In [31]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""
    augmentation_seq = iaa.Sequential([
        # iaa.Affine(rotate=(9, 10)),  # Rotate images by -10 to +10 degrees
        iaa.Multiply((0.8, 1.0)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(gamma=(2.0, 2.2)),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Fliplr(),  # Flip images horizontally with a 50% chance
        # iaa.Multiply((0.8, 1.2)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Affine(rotate=(0, 10)),  # Rotate images by -10 to +10 degrees
        # iaa.GaussianBlur(sigma=(0, 1.0)),  # Apply Gaussian blur with a sigma between 0 and 1.0
        # iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add Gaussian noise with a scale of 0 to 0.05*255
    ])
    for dirname, _, filenames in os.walk('./jaffe'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        hog_descriptors = hog_extraction(extracted_face)
        try:  
            # Calculate the Z-scores for each descriptor
            zscore_hog_descriptors = (hog_descriptors - np.mean(hog_descriptors)) / np.std(hog_descriptors)
            # Fusion using weighted sum
            hog_descriptors = weight2 * zscore_hog_descriptors   
        except Exception as e:
            # print(str(e), np.std(hog_descriptors))
            pass    
        hog_descriptors = np.resize(hog_descriptors, (desired_length,))

        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))

        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            # print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        sift_descriptors = sift_extraction(extracted_face)
        try:
            # Calculate the Z-scores for each descriptor
            zscore_sift_descriptors = (sift_descriptors - np.mean(sift_descriptors)) / np.std(sift_descriptors)
            # Fusion using weighted sum
            sift_descriptors = weight4 * zscore_sift_descriptors   
        except Exception as e:
            # print(str(e))
            pass    
        sift_descriptors = np.resize(sift_descriptors, (desired_length,))

        # fused_features = lbp_descriptors + hog_descriptors + sift_descriptors
        fused_features = hog_descriptors
        if lbp_descriptors is not None:
            fused_features += lbp_descriptors
        # if sift_descriptors is not None:
        #     fused_features += sift_descriptors.astype(np.float64)

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def hog_extraction(extracted_face):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

    # Define HOG parameters
    orientations = 9  # Number of gradient orientations
    pixels_per_cell = (8, 8)  # Cell size in pixels
    cells_per_block = (2, 2)  # Number of cells in each block

    # Compute HOG features
    hog_image, hog_features = hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)    
    # print(hog_features)
    disply_feature("HOG", hog_features)
    return hog_features

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

# Scale-Invariant Feature Transform descriptor
def sift_extraction(extracted_face):
    # Create a SIFT object
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None)
    # print(descriptors)
    disply_feature("SIFT", image_with_keypoints)
    return descriptors

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.034, random_state=44)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(" 2_without_orb_sift_algo algorithm ", source_data_label)
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"2_without_orb_sift_algo_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


 2_without_orb_sift_algo algorithm  jaffe
Accuracy: 0.75
Precision: 0.75
Recall: 0.8611111111111112
F1 Score: 0.6888888888888888


In [30]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""
    augmentation_seq = iaa.Sequential([
        # iaa.Affine(rotate=(9, 10)),  # Rotate images by -10 to +10 degrees
        iaa.Multiply((0.8, 1.0)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(gamma=(2.0, 2.2)),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Fliplr(),  # Flip images horizontally with a 50% chance
        # iaa.Multiply((0.8, 1.2)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Affine(rotate=(0, 10)),  # Rotate images by -10 to +10 degrees
        # iaa.GaussianBlur(sigma=(0, 1.0)),  # Apply Gaussian blur with a sigma between 0 and 1.0
        # iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add Gaussian noise with a scale of 0 to 0.05*255
    ])
    for dirname, _, filenames in os.walk('./jaffe'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        hog_descriptors = hog_extraction(extracted_face)
        try:  
            # Calculate the Z-scores for each descriptor
            zscore_hog_descriptors = (hog_descriptors - np.mean(hog_descriptors)) / np.std(hog_descriptors)
            # Fusion using weighted sum
            hog_descriptors = weight2 * zscore_hog_descriptors   
        except Exception as e:
            # print(str(e), np.std(hog_descriptors))
            pass    
        hog_descriptors = np.resize(hog_descriptors, (desired_length,))

        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))

        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            # print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        sift_descriptors = sift_extraction(extracted_face)
        try:
            # Calculate the Z-scores for each descriptor
            zscore_sift_descriptors = (sift_descriptors - np.mean(sift_descriptors)) / np.std(sift_descriptors)
            # Fusion using weighted sum
            sift_descriptors = weight4 * zscore_sift_descriptors   
        except Exception as e:
            # print(str(e))
            pass    
        sift_descriptors = np.resize(sift_descriptors, (desired_length,))

        # fused_features = lbp_descriptors + hog_descriptors + sift_descriptors
        fused_features = hog_descriptors
        if lbp_descriptors is not None:
            fused_features += lbp_descriptors
        if sift_descriptors is not None:
            fused_features += sift_descriptors.astype(np.float64)

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def hog_extraction(extracted_face):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

    # Define HOG parameters
    orientations = 9  # Number of gradient orientations
    pixels_per_cell = (8, 8)  # Cell size in pixels
    cells_per_block = (2, 2)  # Number of cells in each block

    # Compute HOG features
    hog_image, hog_features = hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)    
    # print(hog_features)
    disply_feature("HOG", hog_features)
    return hog_features

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

# Scale-Invariant Feature Transform descriptor
def sift_extraction(extracted_face):
    # Create a SIFT object
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None)
    # print(descriptors)
    disply_feature("SIFT", image_with_keypoints)
    return descriptors

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.034, random_state=44)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(" 3_without_orb_algo algorithm ", source_data_label)
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"3_without_orb_algo_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


 3_without_orb_algo algorithm  jaffe
Accuracy: 0.75
Precision: 0.75
Recall: 0.861111111111111
F1 Score: 0.6888888888888888


In [28]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""
    augmentation_seq = iaa.Sequential([
        # iaa.Affine(rotate=(9, 10)),  # Rotate images by -10 to +10 degrees
        iaa.Multiply((0.8, 1.0)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(gamma=(2.0, 2.2)),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Fliplr(),  # Flip images horizontally with a 50% chance
        # iaa.Multiply((0.8, 1.2)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Affine(rotate=(0, 10)),  # Rotate images by -10 to +10 degrees
        # iaa.GaussianBlur(sigma=(0, 1.0)),  # Apply Gaussian blur with a sigma between 0 and 1.0
        # iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add Gaussian noise with a scale of 0 to 0.05*255
    ])
    for dirname, _, filenames in os.walk('./jaffe'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.2  # Weight for descriptors sift
            
        hog_descriptors = hog_extraction(extracted_face)
        try:  
            # Calculate the Z-scores for each descriptor
            zscore_hog_descriptors = (hog_descriptors - np.mean(hog_descriptors)) / np.std(hog_descriptors)
            # Fusion using weighted sum
            hog_descriptors = weight2 * zscore_hog_descriptors   
        except Exception as e:
            # print(str(e), np.std(hog_descriptors))
            pass    
        hog_descriptors = np.resize(hog_descriptors, (desired_length,))

        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))

        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            # print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        sift_descriptors = sift_extraction(extracted_face)
        try:
            # Calculate the Z-scores for each descriptor
            zscore_sift_descriptors = (sift_descriptors - np.mean(sift_descriptors)) / np.std(sift_descriptors)
            # Fusion using weighted sum
            sift_descriptors = weight4 * zscore_sift_descriptors   
        except Exception as e:
            # print(str(e))
            pass    
        sift_descriptors = np.resize(sift_descriptors, (desired_length,))

        # fused_features = lbp_descriptors + hog_descriptors + sift_descriptors
        fused_features = hog_descriptors
        if lbp_descriptors is not None:
            fused_features += lbp_descriptors
        # if sift_descriptors is not None:
        #     fused_features += sift_descriptors.astype(np.float64)
        if orb_descriptors is not None:
            fused_features += orb_descriptors.astype(np.float64)

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def hog_extraction(extracted_face):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

    # Define HOG parameters
    orientations = 9  # Number of gradient orientations
    pixels_per_cell = (8, 8)  # Cell size in pixels
    cells_per_block = (2, 2)  # Number of cells in each block

    # Compute HOG features
    hog_image, hog_features = hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)    
    # print(hog_features)
    disply_feature("HOG", hog_features)
    return hog_features

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

# Scale-Invariant Feature Transform descriptor
def sift_extraction(extracted_face):
    # Create a SIFT object
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None)
    # print(descriptors)
    disply_feature("SIFT", image_with_keypoints)
    return descriptors

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.024, random_state=41)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(" 3_without_sift_algo algorithm ", source_data_label)
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"3_without_sift_algo_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


 3_without_sift_algo algorithm  jaffe
Accuracy: 0.5
Precision: 0.7142857142857143
Recall: 0.6428571428571429
F1 Score: 0.38095238095238093


In [1]:
print("kkkkkkkkkkkkkkkkk")

kkkkkkkkkkkkkkkkk


In [64]:
import os
from operator import le
import cv2
import dlib
from time import time
import matplotlib.pyplot as plt
import numpy as np
from imutils import face_utils
from skimage.feature import hog, local_binary_pattern
import mahotas as mh
import joblib
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import imgaug.augmenters as iaa

# Resize descriptors to a consistent length
desired_length = 1000
# Initialize the label variable
img_label = []
feature_descriptors_fused = []


def file_looper_transformer_extract_features_and_train():
    source_data_label = ""
    augmentation_seq = iaa.Sequential([
        # iaa.Affine(rotate=(9, 10)),  # Rotate images by -10 to +10 degrees
        iaa.Multiply((0.8, 1.0)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(gamma=(2.0, 2.2)),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Fliplr(),  # Flip images horizontally with a 50% chance
        # iaa.Multiply((0.8, 1.2)),  # Multiply pixel values by a factor between 0.8 and 1.2
        # iaa.GammaContrast(),  # Adjust gamma contrast between 0.8 and 1.2
        # iaa.ElasticTransformation(alpha=50, sigma=5)  # Apply elastic transformations
        # iaa.Affine(rotate=(0, 10)),  # Rotate images by -10 to +10 degrees
        # iaa.GaussianBlur(sigma=(0, 1.0)),  # Apply Gaussian blur with a sigma between 0 and 1.0
        # iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add Gaussian noise with a scale of 0 to 0.05*255
    ])
    for dirname, _, filenames in os.walk('./jaffe'):
        for filename in filenames:
            # print(os.path.join(dirname, filename))
            img_name, img_extention = os.path.splitext(filename)
            expression_label = (os.path.join(dirname, filename)).split(os.sep)[2]
            if source_data_label == '':
                source_data_label = (os.path.join(dirname, filename)).split(os.sep)[1]

            if(img_extention == ".jpg" or img_extention == ".png"):
                image = cv2.imread(os.path.join(dirname, filename))
                # Apply data augmentation
                image = augmentation_seq(image=image)
                
                ext_feature, the_label = face_detection_extraction(image, expression_label)
                feature_descriptors_fused.append(ext_feature)
                img_label.append(expression_label)


    # return feature_descriptors_fused, img_label
    svmc(feature_descriptors_fused, img_label, source_data_label)


def face_detection_extraction(image, image_label):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray, 1)
    feature_list = []  # List to store the extracted features
    label_list = []  # List to store the corresponding labels

    for face in faces:
        landmarks = predictor(gray, face)
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        extracted_face = image[y:y+h, x:x+w]
        extracted_face = np.array(extracted_face)

        shape = face_utils.shape_to_np(landmarks)
        # Draw on our image, all the found 68 coordinate points (x,y)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        weight1 = 0.9  # Weight for descriptors lbp
        weight2 = 0.9  # Weight for descriptors hog
        weight3 = 0.2  # Weight for descriptors orb
        weight4 = 0.9  # Weight for descriptors sift
            
        hog_descriptors = hog_extraction(extracted_face)
        try:  
            # Calculate the Z-scores for each descriptor
            zscore_hog_descriptors = (hog_descriptors - np.mean(hog_descriptors)) / np.std(hog_descriptors)
            # Fusion using weighted sum
            hog_descriptors = weight2 * zscore_hog_descriptors   
        except Exception as e:
            # print(str(e), np.std(hog_descriptors))
            pass    
        hog_descriptors = np.resize(hog_descriptors, (desired_length,))

        lbp_descriptors = lbp_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        zscore_lbp_descriptors = (lbp_descriptors - np.mean(lbp_descriptors)) / np.std(lbp_descriptors)
        # Fusion using weighted sum
        lbp_descriptors = weight1 * zscore_lbp_descriptors 
        lbp_descriptors = np.resize(lbp_descriptors, (desired_length,))

        orb_descriptors = orb_extraction(extracted_face)
        # Calculate the Z-scores for each descriptor
        # print(" >>>", np.std(orb_descriptors))
        # print(" <  > ", orb_descriptors - np.mean(orb_descriptors) )
        try:
            zscore_orb_descriptors = (orb_descriptors - np.mean(orb_descriptors)) / np.std(orb_descriptors)
            # Fusion using weighted sum
            orb_descriptors = weight3 * zscore_orb_descriptors 
            # print("orb_descriptors ", orb_descriptors)
        except Exception as e:
            # print(str(e))
            pass
        orb_descriptors = np.resize(orb_descriptors, (desired_length,))

        sift_descriptors = sift_extraction(extracted_face)
        try:
            # Calculate the Z-scores for each descriptor
            zscore_sift_descriptors = (sift_descriptors - np.mean(sift_descriptors)) / np.std(sift_descriptors)
            # Fusion using weighted sum
            sift_descriptors = weight4 * zscore_sift_descriptors   
        except Exception as e:
            # print(str(e))
            pass    
        sift_descriptors = np.resize(sift_descriptors, (desired_length,))

        # fused_features = lbp_descriptors + hog_descriptors + sift_descriptors
        fused_features = hog_descriptors
        if lbp_descriptors is not None:
            fused_features += lbp_descriptors
        if sift_descriptors is not None:
            fused_features += sift_descriptors.astype(np.float64)

        # fused_features = np.concatenate((hog_descriptors, lbp_descriptors, sift_descriptors))

        if len(fused_features) > 0:
            feature_list.append(fused_features)
            label_list.append(image_label)  # Add the label to the list

    return feature_list, label_list

        # print(fused_features)
        
        # Display the extracted faces
        # cv2.imshow("Landmark localization", extracted_face)
        # plt.imshow(pixels)
        # plt.axis('off')
        # plt.show()

def hog_extraction(extracted_face):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)

    # Define HOG parameters
    orientations = 9  # Number of gradient orientations
    pixels_per_cell = (8, 8)  # Cell size in pixels
    cells_per_block = (2, 2)  # Number of cells in each block

    # Compute HOG features
    hog_image, hog_features = hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, visualize=True)    
    # print(hog_features)
    disply_feature("HOG", hog_features)
    return hog_features

def lbp_extraction(extracted_face):
    # Compute LBP features

    # Get the dimensions of the image
    # height, width, channels = extracted_face.shape

    # Compute LBP features
    radius = 3
    n_points = 8 * radius
    # convert to 2 dimentional
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    lbp_features = local_binary_pattern(extracted_face, n_points, radius, method='default')
    # print(lbp_features)
    disply_feature("LBP", lbp_features)
    return lbp_features

# Scale-Invariant Feature Transform descriptor
def sift_extraction(extracted_face):
    # Create a SIFT object
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None)
    # print(descriptors)
    disply_feature("SIFT", image_with_keypoints)
    return descriptors

def orb_extraction(extracted_face):
    # Create an ORB object with custom parameters
    orb = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8, edgeThreshold=15)
    # Resize the extracted face to a larger size for better feature extraction
    extracted_face = cv2.resize(extracted_face, (250, 250))
    # Convert the face image to grayscale
    extracted_face = cv2.cvtColor(extracted_face, cv2.COLOR_BGR2GRAY)
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(extracted_face, None)
    # Draw keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(extracted_face, keypoints, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    # disply_feature("ORB", image_with_keypoints)
    return descriptors

def disply_feature(features_type, features):
    # cv2.imshow(features_type, features)
    # # plt.imshow( features)
    # plt.axis('off')
    # plt.show()
    pass

# Support Vector Machine Classifier
def svmc(fused_features, img_labels, source_data_label):
    
    # Create arrays to store the features and labels
    fused_features = np.array(fused_features)
    img_labels = np.array(img_labels)
    
    # Reshape the fused features to have two dimensions
    # fused_features = fused_features.reshape(-1, fused_features.shape[2])
    fused_features = fused_features.reshape(fused_features.shape[0], -1)
 
    # Normalize the feature vectors
    scaler = StandardScaler()
    fused_features = scaler.fit_transform(fused_features)

    # Handle missing values with an imputer transformer
    imputer = SimpleImputer(strategy='mean')
    fused_features = imputer.fit_transform(fused_features)

    # Split the fused features data into training and testing sets
    # X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.2, random_state=41)
    X_train, X_test, y_train, y_test = train_test_split(fused_features, img_labels, test_size=0.002, random_state=41)

    # Create an SVM classifier
    # instantiate classifier with linear kernel and C=100
    svc = svm.SVC(kernel='linear', C=100)

    # Train the classifier
    svc.fit(X_train, y_train)

    # Predict labels for the test set
    y_pred = svc.predict(X_test)
    # print(y_pred)
    # print("< ---------------- >")
    # print(y_test)
    # print("< ---------------- >")
    # print(X_train)
    # print("< ---------------- >")
    # print(X_test)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(" 3_without_orb_algo algorithm ", source_data_label)
    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

   
    joblib.dump(svc, f"3_train_without_orb_algo_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"lbp_zscore_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"hog_on_svm_classifier_{source_data_label}.joblib")
    # joblib.dump(svc, f"sift_on_svm_classifier_{source_data_label}.joblib")

file_looper_transformer_extract_features_and_train()


 3_without_orb_algo algorithm  jaffe
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [ ]:
  3_without_orb_algo algorithm  jaffedtrain
Accuracy: 0.6666666666666666
Precision: 0.6666666666666666
Recall: 0.8333333333333334
F1 Score: 0.5555555555555555